In [5]:
import binascii

def text_to_hexstr(text):
    text_in_bytes = str.encode(text)
    hexstr = str( binascii.hexlify(text_in_bytes) ,'ascii')
    return hexstr

def text_to_felt(text):
    hexstr = text_to_hexstr(text)
    return int(hexstr, 16)

def felt_to_text(felt):
    hexstr = hex(felt)[2:] # remove leading 0x
    bytestr = bytes.fromhex(hexstr)
    return bytestr.decode("utf-8") 

In [6]:
felt = text_to_felt('wala')
text = felt_to_text(felt)
assert text == 'wala'

In [7]:
text_to_hexstr('lsker;aelwkmnfds,ckv')

'6c736b65723b61656c776b6d6e6664732c636b76'

In [8]:
def convert_text_to_felt_array(text):
    '''
    last element of the felt array is an integer indicating the hex-length of the last felt value
    '''
    
    hexstring = text_to_hexstr(text)
    
    arr = []
    s = hexstring
    while( len(s)>62 ):
        felt_hex = s[0:62]
        arr.append( int(felt_hex, 16) )
        s = s[62:]
    felt_hex = s
    last_length = len(felt_hex)
    arr.append( int(felt_hex, 16) )
    arr.append(last_length)

    return arr

def payload_function_from_text (text):
    arr = convert_text_to_felt_array(text)
    func = []
    
    func.append( '@view')
    func.append( 'func payload_hexstring {range_check_ptr} () -> (arr_len : felt, arr : felt*):')
    func.append( '    alloc_locals\n')
    func.append( '    let (local arr) = alloc()')
    for i,ele in enumerate(arr):
        func.append(f'    assert [arr+{i}] = {ele}')
    func.append(f'\n    return ({len(arr)}, arr)')
    func.append( 'end')
    
    for line in func:
        print(line)
    #return func

In [9]:
payload_function_from_text('I am so stoked!')

@view
func payload_hexstring {range_check_ptr} () -> (arr_len : felt, arr : felt*):
    alloc_locals

    let (local arr) = alloc()
    assert [arr+0] = 379694426685219972803782586816357409
    assert [arr+1] = 30

    return (2, arr)
end


In [10]:
def felt_array_into_text (felt_array):
    hexstring = ''
    last_length = felt_array[-1]
    for felt in felt_array[:-2]:
        hexstr = hex(felt)[2:]
        hexstr = hexstr.rjust(62, '0')
        hexstring += hexstr
    hexstr = hex(felt_array[-2])[2:]
    hexstr = hexstr.rjust(last_length, '0')
    hexstring += hexstr
    
    text = bytestr = bytes.fromhex(hexstr)
    return bytestr.decode("utf-8") 
    
    #return hexstring

In [11]:
arr = [379694426685219972803782586816357409, 30]
felt_array_into_text(arr)

'I am so stoked!'

In [15]:
felt = text_to_felt('十年炒币两茫茫')
felt_to_text(felt)

'十年炒币两茫茫'